<a href="https://colab.research.google.com/github/tanyagupta12/weather_forecasting-on-cloud/blob/main/cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load dataset
data = pd.read_csv('/content/Weather Data.csv')

# Select features
selected_features = ['Temp_C', 'Dew Point Temp_C', 'Rel Hum_%', 'Wind Speed_km/h', 'Visibility_km', 'Press_kPa']

# Scale numerical features
scaler = MinMaxScaler()
numeric_data = data[selected_features]
scaled_data = scaler.fit_transform(numeric_data)

# Perform one-hot encoding for the "Weather" column
scaled_data = np.concatenate([scaled_data, pd.get_dummies(data['Weather'])], axis=1)

# Define parameters
sequence_length = 10  # Length of input sequences
n_features = scaled_data.shape[1]  # Number of input features
n_units = 50  # Number of units in the LSTM layer

# Create sequences
sequences = []
targets = []
for i in range(len(scaled_data) - sequence_length):
    sequences.append(scaled_data[i:i+sequence_length])
    targets.append(scaled_data[i+sequence_length][0])  # Predicting temperature, so select the first feature

# Convert lists to numpy arrays
sequences = np.array(sequences)
targets = np.array(targets)

# Split data into training and testing sets
split = int(0.8 * len(sequences))
X_train, X_test = sequences[:split], sequences[split:]
y_train, y_test = targets[:split], targets[split:]

# Define the RNN model with increased complexity
model = Sequential([
    LSTM(n_units, return_sequences=True, input_shape=(sequence_length, n_features)),
    LSTM(n_units),
    Dense(1)
])

# Compile the model with a smaller learning rate
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model for more epochs
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Make predictions
predictions = model.predict(X_test)


# Create DataFrame for actual and predicted values
actual = pd.DataFrame(y_test, columns=['Actual'])
predicted = pd.DataFrame(predictions, columns=['Predicted'])

# Concatenate actual and predicted values
results = pd.concat([actual, predicted], axis=1)
print(results.head(40))  # Print the first 20 rows

# Round predictions to the nearest integer
rounded_predictions = predictions


tolerance=0.02
# Initialize correct_predictions counter
correct_predictions = 0

# Iterate through each prediction and actual value
for pred, actual in zip(predictions, y_test):
    # Check if the absolute difference between prediction and actual is less than or equal to the tolerance
    if (pred - actual) <= tolerance:
        # If yes, increment correct_predictions by 1
        correct_predictions += 1

print(correct_predictions)
# Calculate total number of predictions
total_predictions = len(y_test)
print(len(y_test))
# Calculate accuracy
accuracy = (correct_predictions / total_predictions) * 100

print("Accuracy:",accuracy,"%")


Epoch 1/10
220/220 [==============================] - 8s 20ms/step - loss: 0.0112 - val_loss: 0.0025
Epoch 2/10
220/220 [==============================] - 3s 15ms/step - loss: 0.0018 - val_loss: 0.0020
Epoch 3/10
220/220 [==============================] - 3s 13ms/step - loss: 0.0015 - val_loss: 0.0017
Epoch 4/10
220/220 [==============================] - 3s 13ms/step - loss: 0.0011 - val_loss: 0.0015
Epoch 5/10
220/220 [==============================] - 3s 15ms/step - loss: 9.9364e-04 - val_loss: 0.0012
Epoch 6/10
220/220 [==============================] - 3s 15ms/step - loss: 7.8863e-04 - val_loss: 0.0011
Epoch 7/10
220/220 [==============================] - 3s 14ms/step - loss: 7.2430e-04 - val_loss: 9.3398e-04
Epoch 8/10
220/220 [==============================] - 3s 13ms/step - loss: 5.8412e-04 - val_loss: 7.6924e-04
Epoch 9/10
220/220 [==============================] - 3s 14ms/step - loss: 5.1414e-04 - val_loss: 6.7018e-04
Epoch 10/10
55/55 [==============================] - 0s 5ms